# model

> Defines data and processing structures.

In [ ]:
#| default_exp model

In [ ]:
#| hide
from nbdev.showdoc import *

::: {.callout-tip title = "New to literate programming?" appearance="simple" collapse="true"}
**Welcome to the inner workings of `neuralactivitycubic`!**

This message is here only for you, as we´d like to share some basic information about this piece of research software with you to facilitate your interactions with it. We´re using a literate programming framework called [nbdev](https://nbdev.fast.ai/) for the development, testing, documentation, and dissemination of `neuralactivitycubic`. We believe the concept of rich annotations and usage examples directly intermixed with the source code (read more about the concept of literate programming for instance [here](https://blog.esciencecenter.nl/literate-programming-in-science-1669094541a7)) holds great value and potential, especially in the context of research software, as it makes it easier for others to understand, (re-)use, and ideally even to adapt or contribute to your code. That being said, some things in here might look a bit confusing to you - conversely especially if you are an experienced developer used to "regular" source code. 

For instance, you´ll regularly find the implementation of a class, like the `Model` here, interrupted by markdown text and maybe even some addtional code cells that serve as usage examples or even represent the implementation of tests, only for the subsequent class methods to continue in code cells below, literally patched to the class using the `@patch` decorator. Just be aware that this may not meet your expectations of what conventional source code "should" look like and maybe you can discover some valuable takeaway for you along the way, too!

One final note before you head on, though: we are ourselves still experimenting a lot with this concept and how it can be used to best effect and are more than happy to engage in discussion or to hear your feedback on this topic. Feel free to drop us a message via GitHub!
:::


Everybody needs some help - and we´re standing on the shoulders of some giants here. Let´s start by importing all dependencies we need:

In [ ]:
#| exports

# Actual functional dependencies:
# external:
from pathlib import Path
from datetime import datetime, timezone
from matplotlib.pyplot import show
import ipywidgets as w
import multiprocessing
from fastcore.basics import patch
import gc

# and internal:
from neuralactivitycubic.datamodels import Config
from neuralactivitycubic.processing import AnalysisJob
from neuralactivitycubic.input import RecordingLoaderFactory, ROILoaderFactory, RecordingLoader, ROILoader, get_filepaths_with_supported_extension_in_dirpath, FocusAreaPathRestrictions

# Finally, some dependencies regarding type hints:
from typing import Callable
from matplotlib.figure import Figure
from matplotlib.axes._axes import Axes

Now let´s get started:

In [ ]:
#| exports

class Logger:

    """
    A simple logging utility class that captures log messages with UTC timestamps,
    allows retrieval and clearing of logs, and supports saving them to a file.
    """
    
    def __init__(self):
        self.logs = []

    def add_new_log(self, message: str) -> None:
        """
        Add a new log message with a UTC timestamp prefix. 
        The timestamp is formatted as 'dd-mm-yy HH:MM:SS.ffffff (UTC)'.
        """
        time_prefix_in_utc = datetime.now(timezone.utc).strftime('%d-%m-%y %H:%M:%S.%f')
        self.logs.append(f'{time_prefix_in_utc} (UTC): {message}')
        print(f'{time_prefix_in_utc} (UTC): {message}')

    def get_logs(self) -> list[str]:
        return self.logs

    def clear_logs(self) -> None:
        self.logs = []

    def save_current_logs(self, save_dir: Path) -> None:
        filepath = save_dir.joinpath('logs.txt')
        with open(filepath , 'w+') as logs_file:
            for log_message in self.logs:
                logs_file.write(f'{log_message}\n')
        print(f'Logs saved to {filepath}')


Next up is the `Model`, which is at the absolute core of `neuralactivitycubic`. After initialization of a `Model` object, there are only two more calls you need to make to run a fullblown analysis with `na3`. You can see this in the implementation of the `api.run_analysis()` wrapper, which actually also represents the way we intend you to use `na3` programmatically.

Therefore, we´ll instead continue gradually diving deeper into the inner workings of this class and let you in on our thoughts, reasons, and intentions of the different aspects.

In [ ]:
#| exports

class Model:

    def __init__(self, 
                 config: Config | str
                ) -> None:
        self.num_processes = multiprocessing.cpu_count()
        self.analysis_job_queue = []
        self.result_directories = []
        self.logs = Logger()
        if isinstance(config, str):
            config = Config(data_source_path=config)
        self.config = config
        self.nwb_metadata = None
        self.gui_enabled = False
        self.callback_view_update_infos = None
        self.callback_view_show_output_screen = None
        self.view_output = None
        self.pixel_conversion = None

Well, let´s actually get started with those two methods that run the fullblown analysis and also represent the highest level functions in the `Model`. They are:

In [ ]:
#|exports

@patch
def create_analysis_jobs(self: Model
                        ) -> None:
    self._ensure_data_from_previous_jobs_was_removed()
    self.add_info_to_logs('Basic configurations for data import validated. Starting creation of analysis job(s)...', True)
    if self.config.batch_mode:
        all_subdir_paths_with_rec_file = self._get_all_subdir_paths_with_rec_file(self.config.data_source_path)
        all_subdir_paths_with_rec_file.sort()
        for idx, subdir_path in enumerate(all_subdir_paths_with_rec_file):
            if self.config.results_filepath:
                result_path = self.config.results_filepath / subdir_path.name
            else:
                result_path = subdir_path
            self._create_analysis_jobs_for_single_rec(subdir_path, result_path)
    else:
        self._create_analysis_jobs_for_single_rec()
    self.add_info_to_logs('All job creation(s) completed.', True, 100.0)

and:

In [ ]:
#|exports

@patch
def run_analysis(self: Model
                ) -> None:
    self._display_configs()
    self.add_info_to_logs('Starting analysis...', True)
    for job_idx in range(len(self.analysis_job_queue)):
        analysis_job = self.analysis_job_queue.pop(0)
        self.add_info_to_logs(f'Starting to process analysis job with index #{job_idx}.')
        analysis_job.run_analysis(self.config)
        self.add_info_to_logs(f'Analysis successfully completed. Continue with creation of results.. ')
        analysis_job.create_results(self.config, self.nwb_metadata)
        self.add_info_to_logs(f'Results successfully created at: {analysis_job.results_dir_path}')
        if self.gui_enabled:
            self.callback_view_show_output_screen()
            with self.view_output:
                activity_overview_fig = analysis_job.activity_overview_plot[0]
                activity_overview_fig.set_figheight(400 * self.pixel_conversion)
                activity_overview_fig.tight_layout()
                show(activity_overview_fig)
        self._save_user_settings_as_json(analysis_job)
        self.result_directories.append(analysis_job.results_dir_path)
        self.add_info_to_logs('Updating all log files to contain all logs as final step. All valid logs files will end with this message.')
        self.logs.save_current_logs(analysis_job.results_dir_path)
    else:
        gc.collect()

Both of them come with a bunch of private methods that are not intended to be called from the outside. The following private methods do the heavy lifting when it comes to the creation of analysis jobs. Since `na3` offers batch mode processing, we have to deal with complex recursive browsing of directories and subdirectories, checking for files that can be handled by `na3` using the various `RecordingLoader` and `ROILoader` or rather their respective factories (checkout the `03_input.ipynb` notebook for implementation) that definitely takes a toll on readability. If you´re nonetheless brave enough, feel free to take a look of how we addressed this challenge:

In [ ]:
#| exports

@patch
def _ensure_data_from_previous_jobs_was_removed(self: Model
                                               ) -> None:
    self.add_info_to_logs('Loading of new source data. All previously created jobs & logs will be deleted.', True)
    self.analysis_job_queue = []
    self.logs.clear_logs()

In [ ]:
#| exports

@patch
#@staticmethod
def _get_all_subdir_paths_with_rec_file(self: Model, 
                                        top_level_dir_path: Path
                                       ) -> list[Path]:
    rec_loader_factory = RecordingLoaderFactory()
    supported_extensions_for_recordings = rec_loader_factory.all_supported_extensions
    all_subdir_paths_that_contain_a_supported_recording_file = []
    for elem in top_level_dir_path.iterdir():
        if not elem.name.startswith('.'):
            if elem.is_dir():
                supported_recording_filepaths = [elem_2 for elem_2 in elem.iterdir() if elem_2.suffix in supported_extensions_for_recordings]
                if len(supported_recording_filepaths) > 0:
                    all_subdir_paths_that_contain_a_supported_recording_file.append(elem)
    return all_subdir_paths_that_contain_a_supported_recording_file

In [ ]:
#| exports

@patch
def _create_analysis_jobs_for_single_rec(self: Model, 
                                         recording_path: Path = None,
                                         result_path: Path = None
                                        ) -> None:
    if recording_path is None:
        if self.config.recording_filepath:
            recording_path = self.config.recording_filepath
        else:
            recording_path = self.config.data_source_path
    self.add_info_to_logs(f'Starting with Job creation(s) for {str(recording_path)}', True)
    recording_loader = self._get_recording_loader(recording_path)


    if self.config.roi_filepath:
        roi_filepath = self.config.roi_filepath
    else:
        roi_filepath = recording_loader.filepath.parent

    if self.config.roi_mode == 'file':
        roi_loaders = self._get_all_roi_loaders(roi_filepath)
    else:
        roi_loaders = None

    if self.config.focus_area_filepath:
        focus_area_filepath = self.config.focus_area_filepath
    else:
        focus_area_filepath = recording_loader.filepath.parent

    if result_path is None:
        if self.config.results_filepath:
            result_path = self.config.results_filepath
        else:
            result_path = recording_loader.filepath.parent

    if self.config.focus_area_enabled:
        focus_area_dir_path = self._get_focus_area_dir_path(focus_area_filepath)
        if focus_area_dir_path is None:
            analysis_job = self._create_single_analysis_job(recording_loader, roi_loaders, result_filepath=result_path)
            self.analysis_job_queue.append(analysis_job)
            self.add_info_to_logs(f'Successfully created a single job for {focus_area_filepath} at queue position: #{len(self.analysis_job_queue)}.', True)
        else:
            all_focus_area_loaders = self._get_all_roi_loaders(focus_area_dir_path)
            assert len(all_focus_area_loaders) > 0, f'Focus Area analysis enabled, but no focus area ROIs could be found. Please revisit your source data and retry!'
            for idx, focus_area_loader in enumerate(all_focus_area_loaders):
                if result_path is None:
                    if self.config.results_filepath:
                        result_path = self.config.results_filepath / focus_area_loader.filepath.stem
                analysis_job_with_focus_area = self._create_single_analysis_job(recording_loader, roi_loaders, focus_area_loader, result_filepath=result_path)
                self.analysis_job_queue.append(analysis_job_with_focus_area)
                job_creation_message = (f'Successfully created {idx + 1} out of {len(all_focus_area_loaders)} job(s) for {recording_path} '
                                        f'at queue position: #{len(self.analysis_job_queue)}.')
                self.add_info_to_logs(job_creation_message, True)
    else:
        analysis_job = self._create_single_analysis_job(recording_loader, roi_loaders, result_filepath=result_path)
        self.analysis_job_queue.append(analysis_job)
        self.add_info_to_logs(f'Successfully created a single job for {recording_path} at queue position: #{len(self.analysis_job_queue)}.', True)
    self.add_info_to_logs(f'Finished Job creation(s) for {recording_path}!', True)

In [ ]:
#| exports

@patch
def _get_recording_loader(self: Model, 
                          source_path: Path
                         ) -> RecordingLoader:
    rec_loader_factory = RecordingLoaderFactory()
    if source_path.is_dir():
        self.add_info_to_logs(f'Looking for a valid recording file in {source_path}...', True)
        valid_filepaths = get_filepaths_with_supported_extension_in_dirpath(source_path, rec_loader_factory.all_supported_extensions, 1)
        if len(valid_filepaths) == 0:
            self.add_info_to_logs(f'Could not find any recording files of supported type at {source_path}!', True)
        elif len(valid_filepaths) >  1:
            filepath = valid_filepaths[0]
            too_many_files_message = (f'Found more than a single recording file of supported type at {source_path}, i.e.: {valid_filepaths}. '
                                      f'However, only a single file was expected. NA3 continues with {filepath} and will ignore the other files.')
            self.add_info_to_logs(too_many_files_message, True)
        else:
            filepath = valid_filepaths[0]
            self.add_info_to_logs(f'Found recording file of supported type at: {filepath}.', True)
    else:
        filepath = source_path
        self.add_info_to_logs(f'Found recording file of supported type at: {filepath}.', True)
    recording_loader = rec_loader_factory.get_loader(filepath)
    return recording_loader

In [ ]:
#| exports

@patch
def _get_all_roi_loaders(self: Model, 
                         data_source_path: Path
                        ) -> list[ROILoader]:
    assert data_source_path.is_dir(), f'You must provide a directory as source data when using ROI mode or enabling Focus Areas. Please revisit your input data and retry.'
    roi_loader_factory = ROILoaderFactory()
    all_filepaths_with_supported_filetype_extensions = get_filepaths_with_supported_extension_in_dirpath(data_source_path, roi_loader_factory.all_supported_extensions)
    all_roi_loaders = [roi_loader_factory.get_loader(roi_filepath) for roi_filepath in all_filepaths_with_supported_filetype_extensions]
    return all_roi_loaders

In [ ]:
#| exports

@patch
def _get_focus_area_dir_path(self: Model, 
                             source_path: Path
                            ) -> Path:
    focus_area_path_restrictions = FocusAreaPathRestrictions()
    supported_dir_names = focus_area_path_restrictions.supported_dir_names
    if source_path.is_dir():
        source_dir_path = source_path
    else:
        source_dir_path = source_path.parent
    dirs_with_valid_name = [elem for elem in source_dir_path.iterdir() if (elem.name in supported_dir_names) & (elem.is_dir() == True)]
    if len(dirs_with_valid_name) == 0:
        no_dir_found_message = (f'You enabled Focus Area but a correspondingly named directory could not be found in {source_dir_path}. '
                                f'Please use one of the following for the name of the directory that contains the Focus Area ROIs: {supported_dir_names}. '
                                'In absence of such a directory, analysis is continued without using the Focus Area mode for this data.')
        self.add_info_to_logs(no_dir_found_message, True)
        focus_area_dir_path = None
    elif len(dirs_with_valid_name) > 1:
        too_many_dirs = (f'More than a single Focus Area directory was found in the following parent directory: {source_dir_path}, i.e.: '
                         f'{dirs_with_valid_name}. However, only the use of a single one that contains all your Focus Area ROIS is '
                         f'currently supported. {dirs_with_valid_name[0]} will be used for this analysis, while the other(s): {dirs_with_valid_name[1:]} '
                         'will be ignored to continue processing.')
        self.add_info_to_logs(too_many_dirs, True)
        focus_area_dir_path = dirs_with_valid_name[0]
    else:
        focus_area_dir_path = dirs_with_valid_name[0]
    return focus_area_dir_path

In [ ]:
#| exports

@patch
def _create_single_analysis_job(self: Model,
                                recording_loader: RecordingLoader,
                                roi_loaders: list[ROILoader] | None,
                                focus_area_loader: ROILoader = None,
                                result_filepath: Path = None
                               ) -> AnalysisJob:
    data_loaders = {'recording': recording_loader}
    if roi_loaders is not None:
        data_loaders['rois'] = roi_loaders
    if focus_area_loader is not None:
        data_loaders['focus_area'] = focus_area_loader
    return AnalysisJob(self.num_processes, data_loaders, result_filepath)

The private methods used by `Model.run_analysis()` are less complex:

In [ ]:
#| exports

@patch
def _display_configs(self: Model
                    ) -> None:
    self.add_info_to_logs('Configurations for Analysis Settings and Result Creation validated successfully.', True)
    self.add_info_to_logs(f'Analysis Settings are:')
    for line in self.config.display_all_attributes():
        self.add_info_to_logs(line)

In [ ]:
#| exports

@patch
def _save_user_settings_as_json(self: Model, 
                                analysis_job: AnalysisJob
                               ) -> None:
    filepath = analysis_job.results_dir_path.joinpath('user_settings.json')
    self.config.recording_filepath = analysis_job.recording.filepath
    if analysis_job.focus_area_enabled:
        self.config.focus_area_filepath = analysis_job.focus_area.filepath
    else:
        self.config.focus_area_filepath = None
    if analysis_job.rois_source == 'file':
        self.config.roi_filepath = [roi.filepath for roi in analysis_job.all_rois]
    with open(filepath, 'w+') as user_settings_json:
        user_settings_json.write(self.config.to_json())

Moreover, you may have already spotted that both the `Model.run_analysis()` as well as some of the private methods in this class are using the `Model.add_info_to_logs()` method that handles rendering of logs, leveraging the `Logger` class implemented at the top of this notebook.

In [ ]:
#| exports

@patch
def add_info_to_logs(self: Model, 
                     message: str, 
                     display_in_gui: bool = False, 
                     progress_in_percent: float | None = None
                    ) -> None:
    self.logs.add_new_log(message)
    if (display_in_gui == True) and (self.gui_enabled == True): 
        self.callback_view_update_infos(message, progress_in_percent)


    def add_info_to_logs(self, message: str, display_in_gui: bool = False, progress_in_percent: float | None = None) -> None:
        self.logs.add_new_log(message)
        if (display_in_gui == True) and (self.gui_enabled == True): 
            self.callback_view_update_infos(message, progress_in_percent)

In addition to the text output, the `Model` can also produce some visual outputs. This is particularly of relevance, when the `Model` is connected to the graphical user interface (GUI) of `neuralactivitycubic`. In that case, the output it generates (e.g. logs, plots, or figures) are rendered in the intended widgets. Since we´re following the [Model-View-Controller](https://en.wikipedia.org/wiki/Model%E2%80%93view%E2%80%93controller) pattern, this connection between the *model* (literally this `Model` class) and the *view* (the GUI implemented in `01_view.ipynb`) is established via the *controller* (the `App` implemented in `00_controller.ipynb`) by calling the following two public methods: 

In [ ]:
#| exports

@patch
def setup_connection_to_update_infos_in_view(self: Model, 
                                             update_infos: Callable
                                            ) -> None:
    """ Allows to configure the widget in the GUI that is used to display the logs. """
    self.callback_view_update_infos = update_infos
    self.gui_enabled = True 

In [ ]:
#| exports

@patch
def setup_connection_to_display_results(self: Model, 
                                        show_output_screen: Callable, 
                                        output: w.Output, 
                                        pixel_conversion: float
                                       ) -> None:
    """ Allows to configure the widget in the GUI that is used to display images, plots, and figures. """
    self.callback_view_show_output_screen = show_output_screen
    self.view_output = output
    self.pixel_conversion = pixel_conversion
    self.gui_enabled = True

For this scenario - i.e. usage of `na3` via the GUI - there is also one final method implemented in the `Model` that allows the user to preview how the configured grid size looks like on the image:

In [ ]:
#| exports

@patch
def preview_window_size(self: Model, 
                        grid_size
                       ) -> tuple[Figure, Axes]:
    job_for_preview = self.analysis_job_queue[0]
    preview_fig, preview_ax = job_for_preview.preview_window_size(grid_size)
    return preview_fig, preview_ax

show_doc(Model.preview_window_size)

Phew - that was it! This is the end of the implementation of the `Model` class. Below you find the test cases we implemented with respect to the implementations in this notebook.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## Testing

### Setup

In [ ]:
from shutil import rmtree
import os
from re import compile

import pandas as pd
from pandas.testing import assert_frame_equal

from neuralactivitycubic.view import WidgetsInterface

In [ ]:
test_filepath = Path('../test_data/00')

test00_filepath = Path('../test_data/00')
test01_filepath = Path('../test_data/01')
parent_test_filepath = Path('../test_data')
example_results_dir = Path('../test_data/00/example_test_results_for_spiking_neuron')
results_filepath = Path('../test_data/00/results_directory')
# we have to split results by own directories due to concurrency issues
results_case01_filepath = Path('../test_data/results/case_01/')
results_case02_filepath = Path('../test_data/results/case_02/')
results_case03_filepath = Path('../test_data/results/case_03/')
results_case04_filepath = Path('../test_data/results/case_04/')
results_case05_filepath = Path('../test_data/results/case_05/')
results_case06_filepath = Path('../test_data/results/case_06/')

def test_correct_model_run():
    correct_config = WidgetsInterface().export_user_settings()
    correct_config.data_source_path = test00_filepath / 'spiking_neuron.avi'
    correct_config.save_single_trace_results = True
    model = Model(correct_config)
    model.create_analysis_jobs()
    model.run_analysis()
    return model.result_directories

def test_correct_model_run_with_custom_results_dir():
    correct_config = WidgetsInterface().export_user_settings()
    correct_config.data_source_path = test00_filepath / 'spiking_neuron.avi'
    correct_config.results_filepath = results_filepath
    model = Model(correct_config)
    model.create_analysis_jobs()
    model.run_analysis()
    return correct_config.results_filepath

# tests with different modes of running analysis

def test_run_grid_focus_mode():
    """
    Test run_analysis function with focus area enabled.
    """
    grid_focus_mode_config = WidgetsInterface().export_user_settings()
    grid_focus_mode_config.data_source_path = test01_filepath
    grid_focus_mode_config.focus_area_enabled = True
    grid_focus_mode_config.results_filepath = results_case01_filepath
    model = Model(grid_focus_mode_config)
    model.create_analysis_jobs()
    model.run_analysis()

    return grid_focus_mode_config.results_filepath

def test_run_file_focus_mode():
    """
    Test run_analysis function with focus area enabled.
    """
    file_focus_mode_config = WidgetsInterface().export_user_settings()
    file_focus_mode_config.data_source_path = test01_filepath
    file_focus_mode_config.focus_area_enabled = True
    file_focus_mode_config.roi_mode = 'file'
    file_focus_mode_config.results_filepath = results_case01_filepath
    model = Model(file_focus_mode_config)
    model.create_analysis_jobs()
    model.run_analysis()

    return file_focus_mode_config.results_filepath

def test_run_grid_batch_mode():
    """
    Test run_analysis function with batch mode enabled.
    """
    grid_batch_mode_config = WidgetsInterface().export_user_settings()
    grid_batch_mode_config.data_source_path = parent_test_filepath
    grid_batch_mode_config.batch_mode = True
    grid_batch_mode_config.results_filepath = results_case02_filepath
    model = Model(grid_batch_mode_config)
    model.create_analysis_jobs()
    model.run_analysis()

    return grid_batch_mode_config.results_filepath

def test_run_file_batch_mode():
    """
    Test run_analysis function with batch mode enabled.
    """
    file_batch_mode_config = WidgetsInterface().export_user_settings()
    file_batch_mode_config.data_source_path = parent_test_filepath
    file_batch_mode_config.batch_mode = True
    file_batch_mode_config.roi_mode = 'file'
    file_batch_mode_config.results_filepath = results_case02_filepath
    model = Model(file_batch_mode_config)
    model.create_analysis_jobs()
    model.run_analysis()

    return file_batch_mode_config.results_filepath

def test_run_grid_focus_batch_mode():
    """
    Test run_analysis function with focus and batch mode enabled.
    """
    grid_focus_batch_mode_config = WidgetsInterface().export_user_settings()
    grid_focus_batch_mode_config.data_source_path = parent_test_filepath
    grid_focus_batch_mode_config.batch_mode = True
    grid_focus_batch_mode_config.focus_area_enabled = True
    grid_focus_batch_mode_config.results_filepath = results_case03_filepath
    model = Model(grid_focus_batch_mode_config)
    model.create_analysis_jobs()
    model.run_analysis()

    return grid_focus_batch_mode_config.results_filepath

def test_run_file_focus_batch_mode():
    """
    Test run_analysis function with focus and batch mode enabled.
    """
    file_focus_batch_mode_config = WidgetsInterface().export_user_settings()
    file_focus_batch_mode_config.data_source_path = parent_test_filepath
    file_focus_batch_mode_config.batch_mode = True
    file_focus_batch_mode_config.focus_area_enabled = True
    file_focus_batch_mode_config.roi_mode = 'file'
    file_focus_batch_mode_config.results_filepath = results_case03_filepath
    model = Model(file_focus_batch_mode_config)
    model.create_analysis_jobs()
    model.run_analysis()

    return file_focus_batch_mode_config.results_filepath


In [ ]:
def _test_csv_files(relative_filepath_to_csv: str, results_dir: Path) -> bool:
    filepath = results_dir / relative_filepath_to_csv
    # confirm results have been created:
    if not filepath.is_file():
        return False
    # confirm computational consistency of results, while allowing minor numerical tolerance
    df_test = pd.read_csv(filepath)
    df_validation = pd.read_csv(example_results_dir / relative_filepath_to_csv)
    if assert_frame_equal(df_test, df_validation) is not None:
        return False
    else:
        return True

def test_all_peak_results(results_dir):
    return _test_csv_files('all_peak_results.csv', results_dir)

def test_amplitude_and_df_over_f_results(results_dir):
    return _test_csv_files('Amplitude_and_dF_over_F_results.csv', results_dir)

def test_auc_results(results_dir):
    return _test_csv_files('AUC_results.csv', results_dir)

def test_variance_area_results(results_dir):
    return _test_csv_files('Variance_area_results.csv', results_dir)

def test_representative_single_trace_results(results_dir):
    return _test_csv_files('single_traces/data_of_ROI_7-10.csv', results_dir)

def test_activity_overview_png(results_dir):
    filepath = results_dir / 'activity_overview.png'
    return filepath.is_file()

def test_roi_label_ids_overview_png(results_dir):
    filepath = results_dir / 'ROI_label_IDs_overview.png'
    return filepath.is_file()

def test_individual_traces_with_identified_events_pdf(results_dir):
    filepath = results_dir / 'Individual_traces_with_identified_events.pdf'
    return filepath.is_file()

def test_logs_txt(results_dir):
    filepath = results_dir / 'logs.txt'
    return filepath.is_file()

def test_user_settings_json(results_dir):
    filepath = results_dir / 'user_settings.json'
    return filepath.is_file()

def test_nwb_export(results_dir):
    filepath = results_dir / 'autogenerated_nwb_file.nwb'

def test_all_correct_files_created(results_dir: Path, test_csv: bool = True, single_trace: bool = False) -> bool:
    """
    Check if all expected files have been created in the results directory.
    """
    # confirm all csv files have been created and are correct:
    if test_csv:
        assert test_all_peak_results(results_dir), 'There is an issue with the "all_peak_results.csv" file!'
        assert test_amplitude_and_df_over_f_results(results_dir)
        assert test_auc_results(results_dir)
        assert test_variance_area_results(results_dir)
        if single_trace:
            assert test_representative_single_trace_results(results_dir)

    # confirm all other result files have been created:
    assert test_activity_overview_png(results_dir)
    assert test_roi_label_ids_overview_png(results_dir)
    assert test_logs_txt(results_dir)
    assert test_user_settings_json(results_dir)
    if single_trace:
        assert test_individual_traces_with_identified_events_pdf(results_dir)

    return True


def find_directories_after_test(base_path):
    """
    Find directories after test with timestamps.
    """
    pattern = compile(r'\d{4}_\d{2}_\d{2}_\d{2}-\d{2}-\d{2}_.+')

    matching_dirs = [
        str(base_path) + d for d in os.listdir(base_path)
        if os.path.isdir(os.path.join(base_path, d)) and pattern.fullmatch(d)
    ]

    return matching_dirs

def delete_directories_after_test(paths_list):
    """
    Delete directories after test.
    """
    for res_dir in find_directories_after_test(paths_list):
        rmtree(res_dir)

Run tests:

In [ ]:
# confirm that model can be executed:
result_directories = test_correct_model_run()

for directory in result_directories:
    assert directory.exists()
    assert test_all_correct_files_created(directory)
    # cleanup
    rmtree(directory)

In [ ]:
# confirm that model can be executed with custom result directory:
results_directory = test_correct_model_run_with_custom_results_dir()

assert results_directory.exists()
# only one directory with analysis files should be created, as there are only one focus area:
assert len(list(results_directory.iterdir())) == 1

for directory in results_directory.iterdir():
    assert test_all_correct_files_created(directory)

# cleanup
rmtree(results_directory)

In [ ]:
# confirm that model can be executed with focus mode:
results_directory = test_run_grid_focus_mode()

assert results_directory.exists()

# only one directory should be created, as there are only one focus area:
assert len(list(results_directory.iterdir())) == 2
for directory in results_directory.iterdir():
    assert test_all_correct_files_created(directory, test_csv=False)

# cleanup
rmtree(results_directory)

In [ ]:
# confirm that model can be executed with batch mode:
results_directory = test_run_grid_batch_mode()

assert results_directory.exists()

assert len(list(results_directory.iterdir())) == 3
for directory in results_directory.iterdir():
    for subdirectory in directory.iterdir():
        assert test_all_correct_files_created(subdirectory, test_csv=False)

# cleanup
rmtree(results_directory)

In [ ]:
# confirm that model can be executed with batch and focus mode:
results_directory = test_run_grid_focus_batch_mode()

assert results_directory.exists()

assert len(list(results_directory.iterdir())) == 3
for directory in results_directory.iterdir():
    # test case with multiple focus areas
    if '01' in directory.name:
        assert len(list(directory.iterdir())) == 2
    for subdirectory in directory.iterdir():
        assert test_all_correct_files_created(subdirectory, test_csv=False)

# cleanup
rmtree(results_directory)

In [ ]:
# confirm that model can be executed with focus mode:
results_directory = test_run_file_focus_mode()

assert results_directory.exists()

# only one directory should be created, as there are only one focus area:
assert len(list(results_directory.iterdir())) == 2
for directory in results_directory.iterdir():
    assert test_all_correct_files_created(directory, test_csv=False)

# cleanup
rmtree(results_directory)

In [ ]:
# confirm that model can be executed with batch mode:
results_directory = test_run_file_batch_mode()

assert results_directory.exists()

assert len(list(results_directory.iterdir())) == 3
for directory in results_directory.iterdir():
    for subdirectory in directory.iterdir():
        assert test_all_correct_files_created(subdirectory, test_csv=False)

# cleanup
rmtree(results_directory)

In [ ]:
# confirm that model can be executed with batch and focus mode:
results_directory = test_run_file_focus_batch_mode()

assert results_directory.exists()

assert len(list(results_directory.iterdir())) == 3
for directory in results_directory.iterdir():
    # test case with multiple focus areas
    if '01' in directory.name:
        assert len(list(directory.iterdir())) == 2
    for subdirectory in directory.iterdir():
        assert test_all_correct_files_created(subdirectory, test_csv=False)

# cleanup
rmtree(results_directory)